In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('CKD.csv')
dataset=pd.get_dummies(dataset,drop_first=True,dtype='int')
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [3]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [4]:
dataset['classification_yes'].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [5]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset[['classification_yes']]

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)
from sklearn.preprocessing import StandardScaler
scx=StandardScaler()
x_train=scx.fit_transform(x_train)
x_test=scx.transform(x_test)

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[5,7],'weights':['uniform','distance'],'algorithm':['auto','kd_tree','ball_tree','brute'],'p':[1,2],'metric':['minkowski']}
grid=GridSearchCV(KNeighborsClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


C:\Anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'kd_tree', 'ball_tree', 'brute'],
                         'metric': ['minkowski'], 'n_neighbors': [5, 7],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [8]:
result=grid.cv_results_
grid_prediction=grid.predict(x_test)

In [9]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)
print(cm)

[[45  0]
 [ 6 69]]


In [10]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print(clf_report)

              precision    recall  f1-score   support

           0       0.88      1.00      0.94        45
           1       1.00      0.92      0.96        75

    accuracy                           0.95       120
   macro avg       0.94      0.96      0.95       120
weighted avg       0.96      0.95      0.95       120



In [11]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print('The f1_macro value for the best parameters {}'.format(grid.best_params_),f1_macro)

The f1_macro value for the best parameters {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 5, 'p': 1, 'weights': 'uniform'} 0.9505208333333334


In [12]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9995555555555555

In [13]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007912,0.002300,0.214947,0.008339,auto,minkowski,5,1,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.982221,0.982221,0.947015,0.982221,0.981894,0.975114,0.014050,1
1,0.007933,0.001025,0.229832,0.010866,auto,minkowski,5,1,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.982221,0.982221,0.947015,0.982221,0.981894,0.975114,0.014050,1
2,0.008358,0.002858,0.124386,0.090596,auto,minkowski,5,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.964572,0.964572,0.929513,0.964572,0.963912,0.957428,0.013960,21
3,0.006285,0.005183,0.051540,0.001220,auto,minkowski,5,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.964572,0.982221,0.929513,0.964572,0.963912,0.960958,0.017180,17
4,0.007998,0.004519,0.047053,0.004329,auto,minkowski,7,1,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.964572,0.947015,0.947015,0.982221,0.981894,0.964543,0.015672,9
5,0.006207,0.004149,0.046103,0.003454,auto,minkowski,7,1,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.964572,0.947015,0.947015,0.982221,0.981894,0.964543,0.015672,9
6,0.000254,0.000507,0.051894,0.002652,auto,minkowski,7,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.964572,0.947015,0.929513,0.964572,0.963912,0.953917,0.013930,25
7,0.003286,0.004204,0.056152,0.004902,auto,minkowski,7,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.964572,0.947015,0.929513,0.964572,0.963912,0.953917,0.013930,25
8,0.004083,0.004935,0.027177,0.007628,kd_tree,minkowski,5,1,uniform,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.982221,0.982221,0.947015,0.982221,0.981894,0.975114,0.014050,1
9,0.007334,0.002089,0.015270,0.002128,kd_tree,minkowski,5,1,distance,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.982221,0.982221,0.947015,0.982221,0.981894,0.975114,0.014050,1
